In [1]:
import cv2
import os
import numpy as np
import shutil
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array

In [2]:
original = [
    r'D:\Download\MV_Coursework\Coursework\Code\dataset_split\original\test',
    r'D:\Download\MV_Coursework\Coursework\Code\dataset_split\original\train',
    r'D:\Download\MV_Coursework\Coursework\Code\dataset_split\original\validation'
]

original_6_2_2 = [
    r'D:\Download\MV_Coursework\Coursework\Code\dataset_split\original_6_2_2\test',
    r'D:\Download\MV_Coursework\Coursework\Code\dataset_split\original_6_2_2\train',
    r'D:\Download\MV_Coursework\Coursework\Code\dataset_split\original_6_2_2\validation'
]

original_8_1_1 = [
    r'D:\Download\MV_Coursework\Coursework\Code\dataset_split\original_8_1_1\test',
    r'D:\Download\MV_Coursework\Coursework\Code\dataset_split\original_8_1_1\train',
    r'D:\Download\MV_Coursework\Coursework\Code\dataset_split\original_8_1_1\validation'
]

RGB_augmented_200_224 = [
    r'D:\Download\MV_Coursework\Coursework\Code\augmented_image\RGB_augmented_200_224\test',
    r'D:\Download\MV_Coursework\Coursework\Code\augmented_image\RGB_augmented_200_224\train',
    r'D:\Download\MV_Coursework\Coursework\Code\augmented_image\RGB_augmented_200_224\validation'
]

RGB_augmented_250_224 = [
    r'D:\Download\MV_Coursework\Coursework\Code\augmented_image\RGB_augmented_250_224\test',
    r'D:\Download\MV_Coursework\Coursework\Code\augmented_image\RGB_augmented_250_224\train',
    r'D:\Download\MV_Coursework\Coursework\Code\augmented_image\RGB_augmented_250_224\validation'

]

RGB_augmented_300_224 = [
    r'D:\Download\MV_Coursework\Coursework\Code\augmented_image\RGB_augmented_300_224\test',
    r'D:\Download\MV_Coursework\Coursework\Code\augmented_image\RGB_augmented_300_224\train',
    r'D:\Download\MV_Coursework\Coursework\Code\augmented_image\RGB_augmented_300_224\validation'
]

Mixed_augmented_300_224 = [
    r'D:\Download\MV_Coursework\Coursework\Code\augmented_image\Mixed_augmented_300_224\test',
    r'D:\Download\MV_Coursework\Coursework\Code\augmented_image\Mixed_augmented_300_224\train',
    r'D:\Download\MV_Coursework\Coursework\Code\augmented_image\Mixed_augmented_300_224\validation'
]
Mixed_augmented_200_224 = [
    r'D:\Download\MV_Coursework\Coursework\Code\augmented_image\Mixed_augmented_200_224\test',
    r'D:\Download\MV_Coursework\Coursework\Code\augmented_image\Mixed_augmented_200_224\train',
    r'D:\Download\MV_Coursework\Coursework\Code\augmented_image\Mixed_augmented_200_224\validation'
]

In [3]:
source_dirs = {
    'Healthy': r'D:\Download\MV_Coursework\Coursework\Code\wheat_leaf\Healthy',
    'Septoria': r'D:\Download\MV_Coursework\Coursework\Code\wheat_leaf\septoria',
    'Stripe_Rust': r'D:\Download\MV_Coursework\Coursework\Code\wheat_leaf\stripe_rust'
}
dest_dir = r'D:\Download\MV_Coursework\Coursework\Code\dataset_split\original_6_2_2'

if os.path.exists(dest_dir):
    shutil.rmtree(dest_dir)

train_ratio = 0.6
validation_ratio = 0.2
test_ratio = 0.2

for set_type in ['train', 'validation', 'test']:
    for class_name in source_dirs:
        os.makedirs(os.path.join(dest_dir, set_type, class_name), exist_ok=True)

for class_name, class_path in source_dirs.items():
    images = os.listdir(class_path)
    np.random.shuffle(images) 

    train_split = int(len(images) * train_ratio)
    validation_split = int(len(images) * (train_ratio + validation_ratio))

    for i, img in enumerate(images):
        if i < train_split:
            set_type = 'train'
        elif i < validation_split:
            set_type = 'validation'
        else:
            set_type = 'test'

        source_path = os.path.join(class_path, img)
        dest_path = os.path.join(dest_dir, set_type, class_name, img)
        shutil.copy(source_path, dest_path)

print("Dataset distribution complete.")

Dataset distribution complete.


In [3]:
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
def resize_images_in_directory(directory, target_size):
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.lower().endswith('.jpg'):
                file_path = os.path.join(root, file)
                image = cv2.imread(file_path)
                if image is not None:
                    resized_image = cv2.resize(image, target_size)
                    cv2.imwrite(file_path, resized_image)
                else:
                    print(f"Failed to load image: {file_path}")
    print('Finished resizing images')

def clear_and_create_directory(directory):
    if os.path.exists(directory):
        shutil.rmtree(directory)
    os.makedirs(directory)

enhancement_method = 0  

def mix_augment(image_path):
    global enhancement_method
    image = cv2.imread(image_path)
    
    if enhancement_method > 3:
        enhancement_method = 0  
        
    if enhancement_method == 0:
        clahe = cv2.createCLAHE(clipLimit=9, tileGridSize=(16, 16))
        clahe_image = clahe.apply(cv2.cvtColor(image, cv2.COLOR_RGB2GRAY))
        augmented_image = cv2.cvtColor(clahe_image, cv2.COLOR_GRAY2RGB)
    elif enhancement_method == 1:
        bilateral = cv2.bilateralFilter(image, 13, 30, 30)
        augmented_image = bilateral
    elif enhancement_method == 2:
        equalized_image = cv2.equalizeHist(cv2.cvtColor(image, cv2.COLOR_RGB2GRAY))
        augmented_image = cv2.cvtColor(equalized_image, cv2.COLOR_GRAY2RGB)
    # elif enhancement_method == 3:
    #     gray_image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    #     augmented_image = cv2.merge([gray_image, gray_image, gray_image])
    elif enhancement_method == 3:
        image = img_to_array(image)  
        image = np.expand_dims(image, axis=0) 
        it = datagen.flow(image, batch_size=1)
        batch = next(it)
        augmented_image = batch[0].astype('uint8')
    else:
        raise Exception('Invalid enhancement method')
        
    enhancement_method += 1
    return augmented_image

def rgb_augment(image_path):
    image = cv2.imread(image_path)
    image = img_to_array(image)  
    image = np.expand_dims(image, axis=0) 
    it = datagen.flow(image, batch_size=1)
    batch = next(it)
    augmented_image = batch[0].astype('uint8')
    return augmented_image



def augment_images(category_directory, output_directory, target_count, augmentation_method):
    clear_and_create_directory(output_directory)

    for category in os.listdir(category_directory):
        category_path = os.path.join(category_directory, category)
        category_output_path = os.path.join(output_directory, category)
        clear_and_create_directory(category_output_path)

        for filename in os.listdir(category_path):
            if os.path.isfile(os.path.join(category_path, filename)):
                shutil.copy(os.path.join(category_path, filename), os.path.join(category_output_path, filename))

        last_dir_name = os.path.basename(os.path.normpath(category_directory))
        
        current_count = len(
            [name for name in os.listdir(category_path) if os.path.isfile(os.path.join(category_path, name))])

        if current_count >= target_count:
            continue  
        
        augment_count = int(target_count) - current_count
        print(f'Dataset: {last_dir_name}, Category: {category}, Current Count: {current_count}, Augment Count: {augment_count}')

        augmented = 1 
        while augment_count > 0: 
            for filename in os.listdir(category_path):
                if augment_count <= 0:
                    break  
    
                augmented_image = None
                image_path = os.path.join(category_path, filename)
                if augmentation_method == 'mix':
                    augmented_image = mix_augment(image_path)  
                elif augmentation_method == 'rgb':
                    augmented_image = rgb_augment(image_path)  
                else:
                    raise Exception('Invalid enhancement method')

                base_filename, file_extension = os.path.splitext(filename)
                new_filename = f'aug_{augmented}{file_extension}'

                cv2.imwrite(os.path.join(category_output_path, new_filename), augmented_image)

                augmented += 1
                augment_count -= 1


In [5]:
resize_images_in_directory(r'D:\Download\MV_Coursework\Coursework\Code\dataset_split\original_6_2_2', (224, 224))

Finished resizing images


In [4]:
# for input_dir, output_dir in zip(original_shuffle, RGB_augmented_300_224):
#     augment_images(input_dir, output_dir, target_sample_count)
augment_images(original[1], Mixed_augmented_300_224[1], 300, 'mix')

Dataset: train, Category: Healthy, Current Count: 71, Augment Count: 229
Dataset: train, Category: Septoria, Current Count: 67, Augment Count: 233
Dataset: train, Category: Stripe_Rust, Current Count: 145, Augment Count: 155
